MAIN SCRIPT

In [ ]:
import numpy as np
from helpers import *
from implementations import *
from clean_data import clean_data
from visualize_features import visualize_features
from cross_validation import *
from poly_reg import *
%load_ext autoreload
%autoreload 2

In [ ]:
# Load Data
path_dataset_train = "train.csv"
yb, input_data, ids = load_csv_data(path_dataset_train)

path_dataset_test = "test.csv"
yb_t, input_data_t, ids_t = load_csv_data(path_dataset_test)

In [ ]:
# Data Cleaning
tx0, tx1, tx2, tx3, y0, y1, y2, y3,ids0, ids1, ids2, ids3 = clean_data(input_data,yb,ids) # Note: Didn't use ids, no use

tx0_t, tx1_t, tx2_t, tx3_t, y0_t, y1_t, y2_t, y3_t, ids0_t, ids1_t, ids2_t, ids3_t= clean_data(input_data_t,yb_t,ids_t, test=True) 

In [ ]:
# Feature Visualization
features = np.array([1,4,5]) # Features to compare
visualize_features(tx0,y0,features,'Feature_comparision')

In [ ]:
# Score the predictions
def find_score(y,y_pred):
    count = 0
    for i in range(len(y_pred)):
        if(y_pred[i] == y[i]):
            count = count + 1       
    return (count/len(y_pred))
    

In [ ]:
def construct_predictions(w0, w1, w2, w3, tx0, tx1, tx2, tx3, ids_t,ids0_t, ids1_t,ids2_t,ids3_t):
# Construct the final vector of the predictions using the ids 

    y_pred0 = predict_labels(w0,tx0)
    y_pred1 = predict_labels(w1,tx1)
    y_pred2 = predict_labels(w2,tx2)
    y_pred3 = predict_labels(w3,tx3)

    y_pred = np.ones(len(ids_t))

    for i in range(len(ids0_t)):
        idx = np.where(ids_t==ids0_t[i])[0]
        y_pred[idx] = y_pred0[i]

    for i in range(len(ids1_t)):
        idx = np.where(ids_t==ids1_t[i])[0]
        y_pred[idx] = y_pred1[i]

    for i in range(len(ids2_t)):
        idx = np.where(ids_t==ids2_t[i])[0]
        y_pred[idx] = y_pred2[i]

    for i in range(len(ids3_t)):
        idx = np.where(ids_t==ids3_t[i])[0]
        y_pred[idx] = y_pred3[i]
        
    return y_pred

ridge regression with cross validation

In [ ]:
# use cross validation to find the best lambda for the ridge regression and find the optimal weigths 

lambda0 = best_lambda(y0, tx0)
lambda1 = best_lambda(y1, tx1)
lambda2 = best_lambda(y2, tx2)
lambda3 = best_lambda(y3, tx3)


In [ ]:
#find best degrees

degree0 = best_degree(y0, tx0, lambda0)
degree1 = best_degree(y1, tx1, lambda1)
degree2 = best_degree(y2, tx2, lambda2)
degree3 = best_degree(y3, tx3, lambda3)

In [ ]:
#build polynomial extensions

tx0_poly = build_poly_f(tx0, int(degree0))
tx1_poly = build_poly_f(tx1, int(degree1))
tx2_poly = build_poly_f(tx2, int(degree2))
tx3_poly = build_poly_f(tx3, int(degree3))

tx0_poly_t = build_poly_f(tx0_t, int(degree0))
tx1_poly_t = build_poly_f(tx1_t, int(degree1))
tx2_poly_t = build_poly_f(tx2_t, int(degree2))
tx3_poly_t = build_poly_f(tx3_t, int(degree3))

In [ ]:
# find weigths using ridge_regression

_,w0 = ridge_regression(y0, tx0_poly, lambda0)
_,w1 = ridge_regression(y1, tx1_poly, lambda1)
_,w2 = ridge_regression(y2, tx2_poly, lambda2)
_,w3 = ridge_regression(y3, tx3_poly, lambda3)

In [ ]:
# construct predictions

y_pred = construct_predictions(w0, w1, w2, w3, tx0_poly_t, tx1_poly_t, tx2_poly_t, tx3_poly_t,ids_t,ids0_t, ids1_t,ids2_t,ids3_t)

In [ ]:
create_csv_submission(ids_t, y_pred, 'submission.csv')